In [177]:
import pandas as pd
import codecs

In [178]:
# open file for analysis
filename = "./data/app_review_bugs_test.csv"
file = codecs.open(filename,"r","utf8")
dt = pd.read_csv(file)
file.close()

In [179]:
# cut out useless fields and rows with ratings less than 4 
#dt_4 = dt[dt['Rating'] < 4]
#dt_4_p = dt_4.drop(['Device', "Device Type", "Date", "AppName", "Language"], axis=1)
dt_4_p=dt

In [180]:
# removing punctuation marks to '.' and conversation to lower case
before_lemm = list(dt_4_p.Review)
separates = u".,!?;:()[]{}\n"
separates_for_remove = u'"\'$@#%^&*+-=\|/№'
for i in range(len(before_lemm)):
    before_lemm[i] = unicode(str(before_lemm[i]), 'utf8')
    for j in separates_for_remove:
        before_lemm[i] = before_lemm[i].replace(j, u" ")
    for j in separates:
        before_lemm[i] = before_lemm[i].replace(j, u" . ")

    before_lemm[i] = before_lemm[i].replace(u" но ", u" . ")
    before_lemm[i] = before_lemm[i].replace(u" а ", u" . ")
    before_lemm[i] = before_lemm[i].replace(u".  что ", u" . ")
    
    before_lemm[i] = before_lemm[i].lower()

In [181]:
# open file for lemmatization
# w[0] - form
# w[1] - lemma

# you need to open archive new_dict.rar for this file
dict_file_name = "utils/new_dict.txt"
file_lemm = open(dict_file_name, "r")
lemm_str = file_lemm.readlines()

lemm_arr_form = []
lemm_arr_orig = []
for string in lemm_str:
    string=unicode(str(string), 'utf8')
    string=string.replace("\n", "")
    w = string.split(" ")
    lemm_arr_form.append(w[0])
    lemm_arr_orig.append(w[1])

file_lemm.close()

In [182]:
# just binary searching
def bin_search(arr, word):
   l=0
   r=len(arr)
   while(r-l > 1):
      m = (r + l) // 2
      if word < arr[m]:
         r=m
      else:
         l=m
   return l if arr[l]==word else -1

def correct_text(text, lemm_arr_form, lemm_arr_orig):
    new = []
    reviews = len(text)
    for i in range(reviews):
        if type(text[i]) != unicode:
            review1 = unicode(str(text[i]), 'utf8')
        else:
            review1 = unicode(text[i])
        words = review1.split(u" ")
        new_words = u""
        for w in words:
            index = bin_search(lemm_arr_form, w)
            if index != -1:
                w=lemm_arr_orig[index]
            new_words += w + u" "
        new.append(new_words)
    return new

In [183]:
# getting lemms for words in text
after_lemmatisation = correct_text(before_lemm, lemm_arr_form, lemm_arr_orig)

In [184]:
out_lemms = open("lemm.txt", "w+")
for i in range(len(after_lemmatisation)):
    out_lemms.write(after_lemmatisation[i].encode("utf-8") + "\n")

In [185]:
# reading noisy words 
filename_noise = "./utils/noise_words.csv"
file_noise = codecs.open(filename_noise,"r","utf8")
noise_words = pd.read_csv(file_noise, sep=" ", names=['idx', 'Freq', 'Word', 'Type'])
noise_words = noise_words.drop(['idx', 'Freq', 'Type'], axis=1)
file_noise.close()

In [186]:
# removing noisy words
n_first_noise_words = 5000
first = noise_words.Word[0:n_first_noise_words]
first_list = list(first)

#to unicode
for i in range(len(first_list)):
    if type(first_list[i]) != unicode:
        first_list[i] = unicode(str(first_list[i]), 'utf8')
first_list.sort()

after_noise = []
indexes = []

removed = 0
not_removed = 0
for i in range(len(after_lemmatisation)):
    words = after_lemmatisation[i].split(u" ")
    new_text = u""
    local_indexes = []
    for j in range(len(words)):
        w = words[j]
        if w.find(u".")>-1 or w == u"":
            continue
        index = bin_search(first_list, w)
        if index == -1:
            new_text += w + u" "
            local_indexes.append(j)
            not_removed +=1
        else:
            removed +=1
    after_noise.append(new_text)
    indexes.append(local_indexes)

print "were words = " + str(removed + not_removed)
print "removed = " + str(removed)

were words = 508496
removed = 359998


In [255]:
# reading key words
key_words_file = "./utils/key_words.csv"
file_key = codecs.open(key_words_file,"r","utf8")
key_words_df = pd.read_csv(file_key, sep=",")
key_words=list(key_words_df["word"])
scores=list(key_words_df["score"])
file_key.close()

In [256]:
key_words_file = "words.txt"
file_key = codecs.open(key_words_file,"r","utf8")
key_words_df = pd.read_csv(file_key, sep=",")
artm_key_words=list(key_words_df["word"])
file_key.close()

In [257]:
for i in range(10):
    print artm_key_words[i]

отзыв
планшет
андроиде
интернет
поменять
счёта
функция
оператор
пополнение
удобство


In [275]:
#alternative method
size = len(after_lemmatisation)
min_threshold = 8.0
for_artm_score = 3.5
const_for_hand = 2.5

all_ = 0
true_bug = 0
true_not_bug = 0
false_bug = 0
bug_is_not_rec = 0
results = []

ratings = list(dt['Rating'])

for i in range(size):
    all_+=1
    lemm = after_lemmatisation[i].split(u" ")
    source = before_lemm[i].split(u" ")
    n_words=len(lemm)
    score = 0.0
    for j in range(n_words):
        if lemm[j].encode("utf-8") in artm_key_words:
            score += for_artm_score
        if lemm[j].encode("utf-8") in key_words:
            idx = key_words.index(lemm[j].encode("utf-8"))
            score += scores[idx]/10 + const_for_hand
    if score > min_threshold and ratings[i]<4:
        true_bug+=1
    elif score <= min_threshold and ratings[i]>3:
        true_not_bug+=1
    elif ratings[i]>3:
        false_bug+=1
    elif ratings[i]<4:
        bug_is_not_rec+=1

print true_not_bug
print true_bug
print all_
print float(true_not_bug + true_bug)/all_
print true_bug / float(true_bug+bug_is_not_rec)
print true_not_bug / float(false_bug+true_not_bug)

30706
9548
52130
0.77218492231
0.694349501854
0.800072956565


In [13]:
# do key words searching
size = len(after_lemmatisation)
points_for_word = 10
results = []

# loop by reviews
for i in range(size):
    lemm = after_lemmatisation[i].split(u" ")
    source = before_lemm[i].split(u" ")
    
    # do score calculation for each word
    n_words=len(lemm)
    scores_for_words=[]
    for j in range(n_words):
        scores_for_words.append(0)
        if j in indexes[i]:
            scores_for_words[j] += points_for_word
        if lemm[j] in key_words:
            idx = key_words.index(lemm[j])
            scores_for_words[j] += scores[idx]
    prev=0
    sentence = []
    
    # compute score for each sentence
    for j in range(n_words):
        if lemm[j] == ".":
            score = 0
            for k in range(prev,j):
                score += scores_for_words[k]
            sentence.append([score, prev, j])           
            prev = j
    if prev < n_words -1:
        score = 0
        for k in range(prev,n_words):
            score += scores_for_words[k]
        sentence.append([score, prev, n_words])
    
    # choose 1 sentense with max score
    max = 0
    idx = 0
    for j in range(len(sentence)):
        if sentence[j][0] > max:
            max = sentence[j][0]
            idx = j
    if len(sentence) == 0:
        continue
    x = sentence[idx][1]
    if x !=0:
        x +=1
    y = sentence[idx][2]
    results.append(" ".join(source[x:y]).encode("utf-8"))

C:\Users\egorsmir\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [14]:
# removing useless spases 
for i in range(len(results)):
    if len(results[i])>0:
        while(results[i][0] == " "):
            results[i] = results[i][1:]
            if len(results[i]) == 0:
                break
    if len(results[i])>0:
        while(results[i][len(results[i])-1] == " "):
            results[i] = results[i][0:len(results[i])-1]
            if len(results[i]) == 0:
                break
    while results[i].find("  ")>-1:
        results[i] = results[i].replace("  ", " ")

In [15]:
# examples of result
for i in range(20,40):
    print after_lemmatisation[i]

один вопрос .  вы приложение собираться переделывать .  .  .  я посол запуск ваше приложение приходиться перезапускать телефон .  чтобы система не висеть .  . у я 4 ядро .  я узкий баланс проверять смской через номер 900 лишь бы приложение не открывать .  поучиться у альфабанка или уралсиба .  у он всё гладкий работать .  
стать неудобный пря загрузка выписка . отрываться дать за деть и приходиться постоянный тыкать . чтобы открыть далеко .  так же нет возможность фильтр нормальный .  только по сумма .  и нет как заявить ни зарплатной ведомость .  ни выставление счет 
посол обновление приложение перестать нравиться .  через wi fi невозможный зайти .  почему если у я дом стоять роутер .  я должный тратить трафик инета .  .  .  полный бред .  
не перевод не мобильный не оплатить .  пока шаблон не создать 
этот приложение только чтобы узнать баланс .  много ничто сделать не мочь . даже телефон оплатить 
писать ошибка 24 .  обьясните что этот такой .  хотеть до этот пользоваться прогой  . 

In [176]:
x = list(dt_4_p.Review)
for i in range(20,30):
    print x[i]
    print "Key words: " + results[i]
    print "-------------------------------------"

До обновления все было очень хорошо!!! Потом пришлось снесни приложение нафиг. Как писали многие пользователи из—за политики конфиденциальности. Контакты и геолокация согласен, но на какой ляк фото и сообщения????????  Господа разработчики, продолжайте в том же духе, отличные разработки. Вы прям гуру продвижения продукции в мусорку.  По поводу ваших слов о том, что это политика многих компаний в гугл плэй маркете, скажите, что это за приложения, а потом покажите рейтинг скачивания этих приложений и рейтинг их использования! Работайте для людей и клиентов в первую очередь. А тотальный контроль это не ваша задача.
Key words: мне после запуска вашего приложения приходится перезапускать телефон
-------------------------------------
Зависело при установке
Key words: стало неудобно при загрузке выписки
-------------------------------------
Здравствуйте!Приложение не работает т.к. не приходит СМС с кодом когда пытаюсь войти.Приложение переустанавливал не помогло,х отя раньше работало на этом 

In [17]:
#to file
out_file_name = "results.txt"
file = open(out_file_name, "w+")

res = dt.copy()

ratings = list(res["Rating"])
reviews = list(res["Review"])

file.write("Review,IsBag,KeyWords\n")


size = len(ratings)
idx = 0
for i in range(size):
    string = str(reviews[i]) + ","
    if ratings[i]>=4:
        string += "false,"
    else:
        string += "true," + results[idx]
        idx+=1
    file.write(string + "\n")
file.close()

In [130]:
for i in range(20,40):
    print after_noise[i]

обновление все снесни приложение нафиг многий пользователь из—за конфиденциальность геолокация ляк фото разработчик разработка гуру продвижение мусорку гугл плэй маркете приложение рейтинг скачивание приложение рейтинг длить тотальный 
пря 
приложение т смс код приложение переустанавливал х отя 
пря логинами пароль сило пола 
все обновиться посол просканировался рута доступ приложение 
стипуха 1 стипуха 5 6к 5 
пря загрузка зависать антивирусе 
андроиде слетать сервер сносить приложение устанавливать заново нибудь исправить 
ужассс обновиться рута прошивка приложение 
загружаться 429 
приложение выплатить пря платеж разъяснение приложение выписка 
optimizaciya ya ebal lagaet vash interfeys uebki 

пря ввод зарегистрировать удалять исправиться 
все рута 
приложение поварачивать планшет приложение 
обработка 
приложение выдаёт андроид 4 2 исправить 
шаблон длить оплата мобильный 
приложение сий функционировать грузить 10 мб пря запуск оповещать вирус местоположение вирус финансы отправка

In [126]:
artm_file_name= "artm.txt"
big_artm_file = open(artm_file_name, "w+")
for i in range(len(after_noise)):
    if len(after_noise[i].encode("utf-8")) == 0:
        continue
    big_artm_file.write(" |text " +  after_noise[i].encode("utf-8") + "\n")
big_artm_file.close()

In [127]:
T=8
model_artm = artm.ARTM(num_topics=T, topic_names=["sbj"+str(i) for i in range(T)], class_ids={"text":1 })
#model_artm.prepare_data(path_out=path_dt, path_dict=dict_file_name, path_stop=stop_file_name)

In [128]:
batch_vectorizer = artm.BatchVectorizer(data_path=artm_file_name, data_format="vowpal_wabbit", target_folder="out_batches", batch_size=2000)
my_dictionary=artm.Dictionary()
my_dictionary.gather(data_path=batch_vectorizer.data_path);

model_artm.cache_theta=True
model_artm.scores.add(artm.PerplexityScore(name='PerplexityScore', dictionary=my_dictionary))
model_artm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore',class_id ='text'))
model_artm.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_artm.scores.add(artm.TopTokensScore(name="top_words", num_tokens=15, class_id='text'))
model_artm.initialize(dictionary=my_dictionary)
model_artm.num_document_passes=5
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=-100, dictionary=my_dictionary))
model_artm.regularizers['SparsePhi'].tau = -1*1e4
model_artm.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)
model_artm.save("my_super_model")



In [129]:
for topic_name in model_artm.topic_names:
    print topic_name + ': ',
    for word in model_artm.score_tracker["top_words"].last_tokens[topic_name]:
        print word,
    print

sbj0:  все длить отличный андроид отзыв супер планшет андроиде айфоне ок устраивать понятный интернет поменять счёта
sbj1:  приложение прошивка бесполезный функционал удалять root ваше ещё обновить кром пользователь android ограничить антивируса обновляться
sbj2:  пароль смс 3 регистрация ввод загружаться код грузиться зависать зарегистрироваться банкомат тупить сбер отображаться исправте
sbj3:  рута удалить антивирус вирус тут подтверждение исправлять сбера устанавливать прога пин отстоять сервис встроить штраф
sbj4:  шаблон сбербанк мобильный оплата оплатить доступ онлайн баланс т ios оплачивать тормозить сайт банка пополнить
sbj5:  посол обновление пря всё вылетать кода деть вводить сервер 4 загрузка отпечаток заново запуск настройка
sbj6:  исправить 2 5 1 запускаться минуть уведомление выкидывать 0 10 подключение неудобный fi wi хрень
sbj7:  платеж разработчик удобный 24 скачать устанавливаться выдаёт скачиваться фигня глючит скачивать функционала бонус интерфейс урезать


In [131]:
phi=model_artm.get_phi()


In [132]:
print phi

                    sbj0      sbj1      sbj2      sbj3      sbj4      sbj5  \
садик           0.000000  0.000000  0.000000  0.000000  0.000748  0.000000   
бесшумный       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
накручивать     0.000287  0.000000  0.000000  0.000000  0.000000  0.000000   
мучаюсь         0.000000  0.000000  0.000577  0.000000  0.000000  0.000000   
монополизм      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
банка           0.000000  0.000000  0.000000  0.000000  0.013046  0.000000   
пря             0.000000  0.000000  0.000000  0.000000  0.000000  0.110028   
недеешься       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
раскрываться    0.000072  0.000000  0.000000  0.000000  0.000000  0.000000   
64              0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
деактивируется  0.000000  0.000000  0.000000  0.000000  0.000000  0.000046   
самсунгпэй      0.000000  0.000000  0.000000  0.000000  0.000000

In [152]:
words = []
for i in range(T):
    words.append(phi["sbj"+str(i)])

In [153]:
for i in range(T):
    print words[i]
    words[i] = words[i].copy().sort_values(inplace=False)

садик             0.000000
бесшумный         0.000000
накручивать       0.000287
мучаюсь           0.000000
монополизм        0.000000
банка             0.000000
пря               0.000000
недеешься         0.000000
раскрываться      0.000072
64                0.000000
деактивируется    0.000000
самсунгпэй        0.000000
осчастливить      0.000036
лять              0.000000
жаба              0.000000
депутатбомж       0.000000
закасячили        0.000000
ппочему           0.000036
проблема？         0.000036
полсекунды        0.000000
украдёт           0.000036
новомодный        0.000000
интерфейс         0.000000
79                0.000000
бардак            0.000327
заглавный         0.000000
5000              0.000000
минуть            0.000000
автоплатежи       0.000000
всё               0.000000
                    ...   
обманывать        0.000036
снесёшь           0.000000
p3000s            0.000000
замóк             0.000000
повесить          0.000000
отличо            0.000036
п

In [170]:
res = []
for i in range(T):
    values = words[i].axes
    size = len(values[0])
    for j in range(30):
        res.append(values[0][size-j-1])

In [171]:
print len(res)
n = []
for i in res:
    if i not in n:
        n.append(i)
print len(n)

240
239


In [175]:
out_file_name = "words.txt"
file = open(out_file_name, "w+")
for i in n:
    file.write(i.encode("utf-8") + "\n")
file.close()